In [2]:
:dep num = { version = "^0.4.3" }

use num::{BigInt, BigUint};
use num::ToPrimitive;
fn factorial(n: u8) -> BigUint {
    (1..=n).map(BigUint::from).product()
}

for i in 3..8 {
    assert_eq!(factorial(10), factorial(i) * (i+1 ..= 10).map(|x| x as usize).product::<usize>());
}

()

In [12]:
/// Считает стационарное распределение вероятностей для пространства
fn stationary_prob_distribution(rho1: f64, rho2: f64, max_c: u8, g: u8) -> Vec<f64> {
    let prob_low = |x| (rho1 + rho2).powf(x as f64) / (factorial(x).to_f64().unwrap());
    let prob_high = |x| (
        prob_low(g) *
        rho1.powi(x as i32 - g as i32) /
        ((g+1) as usize..=x as usize).map(|x| x as f64).product::<f64>()
    );
    let prob = |x| if x<=g {prob_low(x)} else {prob_high(x)};

    let mut v = Vec::with_capacity(max_c as usize);
    let p0 = 1.0/(1..=max_c)
        // .map(|x| (println!("{x}"), x).1)
        .map(prob).sum::<f64>();

    for c in 0..=max_c {
        let res = p0 * prob(c);
        v.push(res);
    }
    v
}

In [13]:
let lambda1 = 80.0;
let lambda2 = 50.0;
let mu = 2.0;
let mu1 = mu;
let mu2 = mu;
let rho1 = lambda1 / mu1;
let rho2 = lambda2 / mu2;
let c = 60;
let g: usize = 40;

let dist = stationary_prob_distribution(rho1, rho2, c, g as u8);
println!("probability distribution: {dist:?}");
println!("Sum should be 1: {}", dist.iter().sum::<f64>());

let avg: f64 = dist.iter().enumerate().map(|(i,v)| (i as f64) * v).sum();
println!("Average requests in flight: {avg}");

let time_blocking_prob1 = dist.last().copied().unwrap();
let time_blocking_prob2 = dist.iter().skip(g).sum::<f64>();
println!("time blocking probability (E1): {time_blocking_prob1}");
println!("time blocking probability (E2): {time_blocking_prob2}");

let req_blocking_prob1 = lambda1 / (lambda1 + lambda2) * time_blocking_prob1;
let req_blocking_prob2 = lambda2 / (lambda1 + lambda2) * time_blocking_prob1;
println!("request blocking probability: {req_blocking_prob1}, {req_blocking_prob2}");

probability distribution: [2.5542482810751626e-26, 1.6602613826988556e-24, 5.395849493771281e-23, 1.1691007236504442e-21, 1.899788675931972e-20, 2.469725278711563e-19, 2.6755357186041938e-18, 2.4844260244181796e-17, 2.018596144839771e-16, 1.4578749934953903e-15, 9.476187457720037e-15, 5.599565315925476e-14, 3.0330978794596327e-13, 1.5165489397298164e-12, 7.041120077317004e-12, 3.0511520335040355e-11, 1.2395305136110144e-10, 4.739381375571525e-10, 1.7114432745119397e-9, 5.854937518067162e-9, 1.9028546933718278e-8, 5.889788336627086e-8, 1.7401647358216393e-7, 4.917856862104632e-7, 1.3319195668200045e-6, 3.4629908737320113e-6, 8.657477184330028e-6, 2.084207470301674e-5, 4.838338770343171e-5, 0.00010844552416286419, 0.00023496530235287238, 0.000492669182352797, 0.001000734276654119, 0.001971143272197507, 0.0037683621380246446, 0.0069983868277600555, 0.01263597621678899, 0.022198336597061736, 0.03797083891602666, 0.0632847315267111, 0.10283768873090554, 0.10032945242039565, 0.09555185944799

In [29]:
:dep plotters = { version = "^0.3.6", default-features = false, features = ["evcxr", "all_series", "all_elements"] }
extern crate plotters;
use plotters::prelude::*;

fn draw_chart(data: &Vec<(f32, f32)>, name: impl ToString) -> plotters::evcxr::SVGWrapper {
    let minx = data.iter().min_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let maxx = data.iter().max_by(|a, b| a.0.partial_cmp(&b.0).unwrap_or(std::cmp::Ordering::Equal)).unwrap().0;
    let miny = data.iter().min_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let maxy = data.iter().max_by(|a, b| a.1.partial_cmp(&b.1).unwrap_or(std::cmp::Ordering::Equal)).unwrap().1;
    let figure = evcxr_figure((640, 480), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
            .caption(name.to_string(), ("Arial", 50).into_font())
            .margin(5)
            .x_label_area_size(30)
            .y_label_area_size(30)
            .build_cartesian_2d(minx..maxx, miny..maxy)?;

        chart.configure_mesh().draw()?;

        chart.draw_series(LineSeries::new(
            data.clone(),
            &RED,
        )).unwrap();

        // chart.configure_series_labels()
        //     .background_style(&WHITE.mix(0.8))
        //     .border_style(&BLACK)
        //     .draw()?;
        Ok(())
    });
    return figure;
}

draw_chart(&dist.iter().enumerate().map(|(x,y)| (x as f32,*y as f32)).collect(), "n vs p_n")

n vs p_n
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.01
 
 
 
0.02
 
 
 
0.03
 
 
 
0.04
 
 
 
0.05
 
 
 
0.06
 
 
 
0.07
 
 
 
0.08
 
 
 
0.09
 
 
 
0.1
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 44,444 54,444 64,444 74,444 84,444 94,444 104,444 114,444 124,444 134,444 144,444 154,444 164,444 174,444 184,444 194,444 204,444 214,444 224,444 234,444 244,444 254,444 264,444 274,444 284,444 294,444 304,444 314,444 324,444 334,444 344,443 354,441 364,437 374,430 384,418 394,397 404,361 414,301 424,206 434,56 444,66 454,84 464,109 474,140 484,174 494,209 504,244 514,277 524,308 534,335 544,359 554,379 564,395 574,408 584,418 594,425 604,431 614,435 624,438 634,440 "/>

In [21]:
let mut block_prob = vec![];
let rho2 = 0.0;
for lambda1 in 1..=100 {
    let rho1 = lambda1 as f64 / &mu1;
    let dist = stationary_prob_distribution(rho1, rho2, c, g as u8);
    block_prob.push((lambda1 as f32, dist.last().cloned().unwrap() as f32));
}
block_prob

[(1.0, 0.0), (2.0, 0.0), (3.0, 0.0), (4.0, 0.0), (5.0, 0.0), (6.0, 0.0), (7.0, 0.0), (8.0, 0.0), (9.0, 3e-45), (10.0, 7.08e-43), (11.0, 1.30241e-40), (12.0, 1.4595237e-38), (13.0, 1.0773724e-36), (14.0, 5.572401e-35), (15.0, 2.1209606e-33), (16.0, 6.1803667e-32), (17.0, 1.424118e-30), (18.0, 2.6655018e-29), (19.0, 4.1445406e-28), (20.0, 5.456323e-27), (21.0, 6.18163e-26), (22.0, 6.11159e-25), (23.0, 5.3371994e-24), (24.0, 4.1607228e-23), (25.0, 2.9224462e-22), (26.0, 1.8646572e-21), (27.0, 1.0886201e-20), (28.0, 5.853086e-20), (29.0, 2.9149796e-19), (30.0, 1.3517102e-18), (31.0, 5.8635773e-18), (32.0, 2.3895297e-17), (33.0, 9.183387e-17), (34.0, 3.34006e-16), (35.0, 1.1533328e-15), (36.0, 3.7920564e-15), (37.0, 1.1903672e-14), (38.0, 3.5763998e-14), (39.0, 1.0307645e-13), (40.0, 2.855849e-13), (41.0, 7.62114e-13), (42.0, 1.9624506e-12), (43.0, 4.884289e-12), (44.0, 1.1768128e-11), (45.0, 2.748846e-11), (46.0, 6.2333604e-11), (47.0, 1.3739679e-10), (48.0, 2.9473526e-10), (49.0, 6.159925

In [22]:
draw_chart(&block_prob, "lambda1 vs E")

0.0
 
 
 
0.002
 
 
 
0.004
 
 
 
0.006
 
 
 
0.008
 
 
 
0.01
 
 
 
0.012
 
 
 
0.014
 
 
 
0.016
 
 
 
0.018
 
 
 
0.02
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,444 47,444 53,444 59,444 65,444 71,444 77,444 83,444 89,444 95,444 101,444 107,444 113,444 119,444 125,444 131,444 137,444 143,444 149,444 156,444 162,444 168,444 174,444 180,444 186,444 192,444 198,444 204,444 210,444 216,444 222,444 228,444 234,444 240,444 246,444 252,444 258,444 264,444 270,444 277,444 283,444 289,444 295,444 301,444 307,444 313,444 319,444 325,444 331,444 337,444 343,444 349,444 355,444 361,444 367,444 373,444 379,444 385,444 391,444 398,444 404,444 410,444 416,444 422,444 428,444 434,444 440,444 446,444 452,444 458,444 464,443 470,443 476,442 482,441 488,440 494,438 500,436 506,434 512,431 519,427 525,422 531,416 537,410 543,401 549,392 555,380 561,367 567,352 573,334 579,315 585,292 591,267 597,239 603,208 609,174 615,137 621,96 627,53 634,5 "/>

In [27]:
let mut avg_req_counts = vec![];
let rho2 = 0.0;
for lambda1 in 1..=100 {
    let rho1 = (lambda1 as f64) / &mu1;
    let dist = stationary_prob_distribution(rho1, rho2, c, g as u8);
    let avg: f64 = dist.iter().enumerate().map(|(i,v)| i as f64 * v).sum();
    avg_req_counts.push((lambda1 as f32, avg as f32));
}
avg_req_counts

[(1.0, 1.2707471), (2.0, 1.5819767), (3.0, 1.9308254), (4.0, 2.3130352), (5.0, 2.7235637), (6.0, 3.157187), (7.0, 3.6089818), (8.0, 4.0746293), (9.0, 4.550552), (10.0, 5.0339184), (11.0, 5.5225697), (12.0, 6.0149093), (13.0, 6.509787), (14.0, 7.006389), (15.0, 7.5041504), (16.0, 8.002685), (17.0, 8.50173), (18.0, 9.001111), (19.0, 9.500711), (20.0, 10.000454), (21.0, 10.500289), (22.0, 11.000184), (23.0, 11.500116), (24.0, 12.000073), (25.0, 12.500047), (26.0, 13.00003), (27.0, 13.500018), (28.0, 14.000011), (29.0, 14.500008), (30.0, 15.000005), (31.0, 15.500003), (32.0, 16.000002), (33.0, 16.500002), (34.0, 17.0), (35.0, 17.5), (36.0, 18.0), (37.0, 18.5), (38.0, 19.0), (39.0, 19.5), (40.0, 20.0), (41.0, 20.5), (42.0, 21.0), (43.0, 21.5), (44.0, 22.0), (45.0, 22.5), (46.0, 23.0), (47.0, 23.5), (48.0, 24.0), (49.0, 24.5), (50.0, 25.0), (51.0, 25.5), (52.0, 26.0), (53.0, 26.5), (54.0, 27.0), (55.0, 27.5), (56.0, 27.999998), (57.0, 28.499998), (58.0, 28.999994), (59.0, 29.499992), (60.0, 

In [28]:
draw_chart(&avg_req_counts, "lambda1 vs Nbar")

5.0
 
 
 
10.0
 
 
 
15.0
 
 
 
20.0
 
 
 
25.0
 
 
 
30.0
 
 
 
35.0
 
 
 
40.0
 
 
 
45.0
 
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
70.0
 
 
 
80.0
 
 
 
90.0
 
 
 
100.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="35,444 41,442 47,438 53,435 59,431 65,427 71,423 77,419 83,414 89,410 95,405 101,401 107,396 113,392 119,387 125,382 131,378 137,373 143,369 149,364 156,359 162,355 168,350 174,346 180,341 186,336 192,332 198,327 204,323 210,318 216,313 222,309 228,304 234,300 240,295 246,290 252,286 258,281 264,277 270,272 277,267 283,263 289,258 295,254 301,249 307,244 313,240 319,235 325,230 331,226 337,221 343,217 349,212 355,207 361,203 367,198 373,194 379,189 385,184 391,180 398,175 404,171 410,166 416,161 422,157 428,152 434,148 440,143 446,138 452,134 458,129 464,125 470,120 476,115 482,111 488,106 494,102 500,97 506,92 512,88 519,83 525,79 531,74 537,70 543,65 549,61 555,57 561,52 567,48 573,44 579,40 585,36 591,32 597,28 603,24 609,20 615,16 621,12 627,9 634,5 "/>